In [1]:
import numpy as np
import pandas as pd
from random import seed
from random import random
import matplotlib.pyplot as plt
import math
import statistics

# State soace {0,1,2}
P = np.array([[0.2, 0.3, 0.5],
              [0.0, 0.3, 0.7],
              [0.5, 0.4, 0.1]])

#Question 1: Produce data using simulation of this Markov chain.
print("Question 1: Produce data using simulation of this Markov chain.")

#This part of the code displays a short sample path sampled from the DTMC
n = 15
start_state = 0
print(start_state, end = " ")
prev_state = start_state
while n:
    curr_state = np.random.choice([0,1,2], p = P[prev_state]) # state space {0, 1, 2}
    print("-->", curr_state, end = " ")
    prev_state=curr_state
    n-=1
print("--> ...")

#This part of the code produces a long sample path (length of sample path is m+1)
m = 40000
start_state = 0
prev_state = start_state
my_data_array = np.zeros(m+1) #sample path array
for i in range(m):
    state = np.random.choice([0,1,2], p = P[prev_state])
    my_data_array[i+1] = state
    prev_state = state
#print(my_data_array)

#Question 2: Provide an estimate for the probability P (X_10 = 2 | X0 = 1).
print("Question 2: Provide an estimate for the probability P (X_10 = 2 | X0 = 1).")
#This part of the code computes the exact result. We use this as baseline to make sure our code is correct
transient_matrix = np.linalg.matrix_power(P, 10) #transient matrix for time=10 (p0,1(10)) 
print("exact value = ",transient_matrix[[0],[1]])

#estimation
n00=0 #Counter for the occurrence of the sequence 0->0 in 10 steps
n01=0 #Counter for the occurrence of the sequence 0->1 in 10 steps / Number of successes
n02=0 #Counter for the occurrence of the sequence 0->0 in 10 steps / n00+n02 = number of failures / n00+n01+n02 = number of runs = sample size

for i in range(m-10):
    if my_data_array[i]==0: #discover all instances that have a zero
        if my_data_array[i+10]==0: #discover all instances that after 10 steps end at a zero
            n00=n00+1 #count the failures of the first type
        elif my_data_array[i+10]==1: #discover all instances that after 10 steps end at a one
            n01=n01+1 #count the successes
        else: #discover all instances that after 10 steps end at a two
            n02=n02+1 #count the failures of the second type

print("for sample size = ", n00+n01+n02, "the estimated probability is equal to ",n01/(n00+n01+n02))

#Question 3: Compute a 95% confidence interval around this estimate. How many runs are required to achieve a half-width of at most 0:01?
print("Question 3: Compute a 95% confidence interval around this estimate. How many runs are required to achieve a half-width of at most 0:01?")
#Exact value for the variance of a Bernoulli random variable
print("exact variance = ",transient_matrix[[0],[1]]*(1-transient_matrix[[0],[1]]))

#Formula for the sample variance
S01=(n01/(n00+n01+n02-1))*((1-(n01/(n00+n01+n02)))**2)+((n00+n02)/(n00+n01+n02-1))*((0-(n01/(n00+n01+n02)))**2)  #Variance
print("sample variance = ",S01)
print("confidence interval = (",(n01/(n00+n01+n02))-1.96*math.sqrt(S01/(n00+n01+n02)),",",(n01/(n00+n01+n02))+1.96*math.sqrt(S01/(n00+n01+n02)),")") #Confidence Interval
print("half width of the confidence interval = ", 1.96*math.sqrt(S01/(n00+n01+n02)))


#Question 4: Repeat 1-3 for the quantity limt\inftyP (Xt = 2 | X0 = 1).
print("Question 4: Repeat 1-3 for the quantity lim_t P (Xt = 2 | X0 = 1).")
#This part of the code computes the exact result. The code is copied from the link https://towardsdatascience.com/markov-chain-analysis-and-simulation-using-python-4507cee0b06e covered on Tuesday
A = np.append(np.transpose(P)-np.identity(3),[[1,1,1]],axis=0)
b = np.transpose(np.array([0,0,0,1]))
pii=np.linalg.solve(np.transpose(A).dot(A), np.transpose(A).dot(b))
print(pii[[1]])

#This part covers the estimation
mt = 40000
start_state = 0
prev_state = start_state
my_data_t_array = np.zeros(mt+1) #sample path array
for i in range(mt):
    state = np.random.choice([0,1,2], p = P[prev_state])
    my_data_t_array[i+1] = state
    prev_state = state
#print(my_data_array)

t=650 #We set the time to a high value. The higher the value for t, the better the estimation for the limit
nt00=0 #Counter for the occurrence of the sequence 0->0 in t steps
nt01=0 #Counter for the occurrence of the sequence 0->1 in t steps / Number of successes
nt02=0 #Counter for the occurrence of the sequence 0->0 in t steps / nt00+nt02 = number of failures / nt00+nt01+nt02 = number of runs = sample size

for i in range(mt-t):
    if my_data_t_array[i]==0: #discover all instances that have a zero
        if my_data_t_array[i+t]==0: #discover all instances that after t steps end at a zero
            nt00=nt00+1 #count the failures of the first type
        elif my_data_t_array[i+t]==1: #discover all instances that after t steps end at a one
            nt01=nt01+1 #count the successes
        else: #discover all instances that after t steps end at a two
            nt02=nt02+1 #count the failures of the second type

print("for sample size = ", nt00+nt01+nt02, "the estimated probability is equal to ",nt01/(nt00+nt01+nt02))

St01=(nt01/(nt00+nt01+nt02-1))*((1-(nt01/(nt00+nt01+nt02)))**2)+((nt00+nt02)/(nt00+nt01+nt02-1))*((0-(nt01/(nt00+nt01+nt02)))**2)  #Variance
print("sample variance = ",St01)
print("confidence interval = (",(nt01/(nt00+nt01+nt02))-1.96*math.sqrt(St01/(nt00+nt01+nt02)),",",(nt01/(nt00+nt01+nt02))+1.96*math.sqrt(St01/(nt00+nt01+nt02)),")") #Confidence Interval
print("half width of the confidence interval = ", 1.96*math.sqrt(St01/(nt00+nt01+nt02)))




#Question 5: Let gamma = 0:99. Repeat 1-3 for the quantity E[sum gamma^tX_t].
print("Question 5: Let gamma = 0.99. Repeat 1-3 for the quantity E[sum gamma^tX_t].")
gamma = 0.99

nlimit = 20000 #number of runs = samples
tlimit = 650 #We set the time to a high value. The higher the value for t, the better the estimation.
print("Note that the terms not included are at most ", (gamma**tlimit)*2/(1-gamma))

start_state = 0
prev_state = start_state
summand=np.zeros(nlimit)
for n in range(nlimit):
    my_gamma_data_limit_array = np.zeros(tlimit) #sample path array
    for i in range(tlimit):
        my_gamma_data_limit_array[i] = prev_state*(gamma**i)+my_gamma_data_limit_array[i-1]
        state = np.random.choice([0,1,2], p = P[prev_state])
        prev_state = state
    summand[n] = my_gamma_data_limit_array[tlimit-1]

print("for sample size = ", nlimit, "the estimated quantity is equal to ",statistics.mean(summand))

print("sample variance = ",statistics.variance(summand))
print("confidence interval = (",statistics.mean(summand)-1.96*math.sqrt(statistics.variance(summand)/nlimit),",",statistics.mean(summand)+1.96*math.sqrt(statistics.variance(summand)/nlimit),")") #Confidence Interval
print("half width of the confidence interval = ", 1.96*math.sqrt(statistics.variance(summand)/nlimit))


Question 1: Produce data using simulation of this Markov chain.
0 --> 1 --> 2 --> 1 --> 1 --> 2 --> 1 --> 1 --> 1 --> 2 --> 0 --> 2 --> 0 --> 2 --> 1 --> 2 --> ...
Question 2: Provide an estimate for the probability P (X_10 = 2 | X0 = 1).
exact value =  [0.3405954]
for sample size =  10102 the estimated probability is equal to  0.3444862403484458
Question 3: Compute a 95% confidence interval around this estimate. How many runs are required to achieve a half-width of at most 0:01?
exact variance =  [0.22459017]
sample variance =  0.22583782631297972
confidence interval = ( 0.33521899348974366 , 0.353753487207148 )
half width of the confidence interval =  0.009267246858702158
Question 4: Repeat 1-3 for the quantity lim_t P (Xt = 2 | X0 = 1).
[0.34057971]
for sample size =  10012 the estimated probability is equal to  0.3436875749101079
sample variance =  0.22558895762004988
confidence interval = ( 0.3343838990828534 , 0.35299125073736237 )
half width of the confidence interval =  0.00930